José est le gestionnaire de la base de données d'une entreprise qui vend des fournitures de<br>
bureau. Il utilise MongoDB pour gérer les informations sur les produits, les ventes, et les<br>
clients. La collection fournitures contient les documents suivants, représentant divers articles<br>
que l'entreprise vend :<br>
[<br>
{ "nom": "Stylo à bille", "marque": "Bic", "stock": 100, "categorie": "Écriture", "prix":<br>
1.25 },<br>
{ "nom": "Cahier A5", "marque": "Oxford", "stock": 50, "categorie": "Papeterie",<br>
"prix": 2.50 },<br>
{ "nom": "Marqueur permanent", "marque": "Sharpie", "stock": 75, "categorie":<br>
"Écriture", "prix": 1.75 },<br>
{ "nom": "Calculatrice scientifique", "marque": "Casio", "stock": 30, "categorie":<br>
"Électronique", "prix": 15.00 },<br>
{ "nom": "Pack de 500 feuilles A4", "marque": "Navigator", "stock": 150, "categorie":<br>
"Papeterie", "prix": 5.00 }<br>
]<br>

# test avec mongoDb en local


executer la commande podman-compose up -d dans un terminal avant de lancer le notebook. 

In [1]:
# import des bibliothèques
from pymongo import MongoClient
import pandas as pd
import numpy as np

In [2]:
# connection a mongoDB
client = MongoClient(host='localhost',port=8090 ,username='utilisateur', password ='motdepasse')

In [3]:
# création database
db_name = client["inventairejosé"]

In [4]:
# création dictionnaire pour envoyer en tant que document
base_dict = ({ "nom": "Stylo à bille", "marque": "Bic", "stock": 100, "categorie": "Écriture", "prix":
1.25 },
{ "nom": "Cahier A5", "marque": "Oxford", "stock": 50, "categorie": "Papeterie",
"prix": 2.50 },
{ "nom": "Marqueur permanent", "marque": "Sharpie", "stock": 75, "categorie":
"Écriture", "prix": 1.75 },
{ "nom": "Calculatrice scientifique", "marque": "Casio", "stock": 30, "categorie":
"Électronique", "prix": 15.00 },
{ "nom": "Pack de 500 feuilles A4", "marque": "Navigator", "stock": 150, "categorie":
"Papeterie", "prix": 5.00 })

In [5]:
#creation collection
collection_jose = db_name["josé_items"]
# supression des documents de la collection pour eviter les doublons
collection_jose.delete_many({})
# insertion des documents disct
collection_jose.insert_many(base_dict)

InsertManyResult([ObjectId('663908ea39820962ca02bacb'), ObjectId('663908ea39820962ca02bacc'), ObjectId('663908ea39820962ca02bacd'), ObjectId('663908ea39820962ca02bace'), ObjectId('663908ea39820962ca02bacf')], acknowledged=True)

# Lecture collection à José

In [6]:
# Lecture des documents en mode texte
find = collection_jose.find()
for ligne in find:
    print(ligne)

{'_id': ObjectId('663908ea39820962ca02bacb'), 'nom': 'Stylo à bille', 'marque': 'Bic', 'stock': 100, 'categorie': 'Écriture', 'prix': 1.25}
{'_id': ObjectId('663908ea39820962ca02bacc'), 'nom': 'Cahier A5', 'marque': 'Oxford', 'stock': 50, 'categorie': 'Papeterie', 'prix': 2.5}
{'_id': ObjectId('663908ea39820962ca02bacd'), 'nom': 'Marqueur permanent', 'marque': 'Sharpie', 'stock': 75, 'categorie': 'Écriture', 'prix': 1.75}
{'_id': ObjectId('663908ea39820962ca02bace'), 'nom': 'Calculatrice scientifique', 'marque': 'Casio', 'stock': 30, 'categorie': 'Électronique', 'prix': 15.0}
{'_id': ObjectId('663908ea39820962ca02bacf'), 'nom': 'Pack de 500 feuilles A4', 'marque': 'Navigator', 'stock': 150, 'categorie': 'Papeterie', 'prix': 5.0}


In [7]:
#vue pandas :
mongodf = pd.DataFrame(list(collection_jose.find()))
mongodf.head(10)

,_id,nom,marque,stock,categorie,prix
0,663908ea39820962ca02bacb,Stylo à bille,Bic,100,Écriture,1.25
1,663908ea39820962ca02bacc,Cahier A5,Oxford,50,Papeterie,2.50
2,663908ea39820962ca02bacd,Marqueur permanent,Sharpie,75,Écriture,1.75
3,663908ea39820962ca02bace,Calculatrice scientifique,Casio,30,Électronique,15.00
4,663908ea39820962ca02bacf,Pack de 500 feuilles A4,Navigator,150,Papeterie,5.00


José a reçu un nouveau stock de "Stylos à bille" et doit mettre à jour la quantité en
stock. Comment augmenterait-il le stock de 50 unités pour cet article?

In [8]:
collection_jose.update_one({'nom':'Stylo à bille'},{'$inc': {'stock': 50}}, upsert=True)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [9]:
mongodf = pd.DataFrame(list(collection_jose.find()))
mongodf.head(10)

,_id,nom,marque,stock,categorie,prix
0,663908ea39820962ca02bacb,Stylo à bille,Bic,150,Écriture,1.25
1,663908ea39820962ca02bacc,Cahier A5,Oxford,50,Papeterie,2.50
2,663908ea39820962ca02bacd,Marqueur permanent,Sharpie,75,Écriture,1.75
3,663908ea39820962ca02bace,Calculatrice scientifique,Casio,30,Électronique,15.00
4,663908ea39820962ca02bacf,Pack de 500 feuilles A4,Navigator,150,Papeterie,5.00


Pour une promotion, José souhaite supprimer les taxes des produits dont le prix est
inférieur à 2 euros. Comment pourrait-il supprimer le champ `prix` de ces produits
spécifiques?

In [10]:
# si vous voulez changer la collection :
# collection_jose.update_many({'prix':{'$lt':2}},{'$unset':{'prix':1}},upsert=False)
# mongodf = pd.DataFrame(list(collection_jose.find()))
# mongodf.head(10)

#juste un affichage :
mongodf.loc[mongodf['prix']<2,'prix'] = np.nan
mongodf.head(10)

,_id,nom,marque,stock,categorie,prix
0,663908ea39820962ca02bacb,Stylo à bille,Bic,150,Écriture,NaN
1,663908ea39820962ca02bacc,Cahier A5,Oxford,50,Papeterie,2.5
2,663908ea39820962ca02bacd,Marqueur permanent,Sharpie,75,Écriture,NaN
3,663908ea39820962ca02bace,Calculatrice scientifique,Casio,30,Électronique,15.0
4,663908ea39820962ca02bacf,Pack de 500 feuilles A4,Navigator,150,Papeterie,5.0


Il a besoin de calculer le stock total disponible pour la catégorie 'Écriture'. Quelle
commande d'agrégation utiliserait-il?

In [11]:
mongodf = pd.DataFrame(list(collection_jose.find()))
mongodf[mongodf['categorie']=='Écriture'].groupby(by=['nom','marque']).sum().reset_index()

,nom,marque,_id,stock,categorie,prix
0,Marqueur permanent,Sharpie,663908ea39820962ca02bacd,75,Écriture,1.75
1,Stylo à bille,Bic,663908ea39820962ca02bacb,150,Écriture,1.25


In [12]:
# aggrégation version mongo
curseur_agreg = collection_jose.aggregate([
    {"$match" :{"categorie":"Écriture"}},
    {"$group": {"_id": ("$categorie","$marque"),
              "stock": {'$sum':'$stock'}}
    }
])

for ligne in curseur_agreg:
    print(ligne)

{'_id': ['Écriture', 'Sharpie'], 'stock': 75}
{'_id': ['Écriture', 'Bic'], 'stock': 150}


José veut ajouter un nouveau produit : `{ "nom": "Ruban adhésif", "marque": "Scotch",
"stock": 40, "categorie": "Papeterie", "prix": 3.75 }`. Quelle commande doit-il exécuter?

In [13]:
dict_ajout = { "nom": "Ruban adhésif", "marque": "Scotch",
"stock": 40, "categorie": "Papeterie", "prix": 3.75 }
collection_jose.insert_one(dict_ajout)

InsertOneResult(ObjectId('663908eb39820962ca02bad0'), acknowledged=True)

In [14]:
mongodf = pd.DataFrame(list(collection_jose.find()))
mongodf.head(10)

,_id,nom,marque,stock,categorie,prix
0,663908ea39820962ca02bacb,Stylo à bille,Bic,150,Écriture,1.25
1,663908ea39820962ca02bacc,Cahier A5,Oxford,50,Papeterie,2.50
2,663908ea39820962ca02bacd,Marqueur permanent,Sharpie,75,Écriture,1.75
3,663908ea39820962ca02bace,Calculatrice scientifique,Casio,30,Électronique,15.00
4,663908ea39820962ca02bacf,Pack de 500 feuilles A4,Navigator,150,Papeterie,5.00
5,663908eb39820962ca02bad0,Ruban adhésif,Scotch,40,Papeterie,3.75


Pour optimiser les recherches par catégorie, quel index José devrait-il créer?<br>
<br>
Je pense qu'il devrais faire un index text avec la colonne categorie


In [15]:
collection_jose.create_index('categorie')

'categorie_1'

Comment José pourrait-il mettre à jour le prix du "Marqueur permanent" pour
augmenter de 10%?

In [16]:
collection_jose.update_one({'nom':'Marqueur permanent'},{'$mul': {'prix': 1.1}}, upsert=True)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [17]:
mongodf = pd.DataFrame(list(collection_jose.find()))
mongodf.head(10)

,_id,nom,marque,stock,categorie,prix
0,663908ea39820962ca02bacb,Stylo à bille,Bic,150,Écriture,1.250
1,663908ea39820962ca02bacc,Cahier A5,Oxford,50,Papeterie,2.500
2,663908ea39820962ca02bacd,Marqueur permanent,Sharpie,75,Écriture,1.925
3,663908ea39820962ca02bace,Calculatrice scientifique,Casio,30,Électronique,15.000
4,663908ea39820962ca02bacf,Pack de 500 feuilles A4,Navigator,150,Papeterie,5.000
5,663908eb39820962ca02bad0,Ruban adhésif,Scotch,40,Papeterie,3.750


Si José veut regrouper tous les produits par marque et savoir combien chaque marque
offre de types de produits, quelle requête d'agrégation devrait-il utiliser?

In [18]:
mongodf = pd.DataFrame(list(collection_jose.find()))
mongodf[['marque','nom']].groupby(by=['marque']).count().reset_index().rename(columns={'nom':'nombre de produit'})

,marque,nombre de produit
0,Bic,1
1,Casio,1
2,Navigator,1
3,Oxford,1
4,Scotch,1
5,Sharpie,1


In [19]:
#version mongo
curseur_agreg = collection_jose.aggregate([
      {'$group':{'_id': '$marque',
              'nb': {'$sum': 1}}}
])

for ligne in curseur_agreg:
    print(ligne)

{'_id': 'Sharpie', 'nb': 1}
{'_id': 'Bic', 'nb': 1}
{'_id': 'Oxford', 'nb': 1}
{'_id': 'Navigator', 'nb': 1}
{'_id': 'Scotch', 'nb': 1}
{'_id': 'Casio', 'nb': 1}


José veut vérifier les stocks bas et prévoir les commandes. Comment pourrait-il lister
tous les produits ayant un stock inférieur à 40 unités?

In [20]:
mongodf[mongodf['stock']<40]

,_id,nom,marque,stock,categorie,prix
3,663908ea39820962ca02bace,Calculatrice scientifique,Casio,30,Électronique,15.0


In [22]:
curseur = collection_jose.find({'stock':{'$lt':40}})

for ligne in curseur:
    print(ligne)

{'_id': ObjectId('663908ea39820962ca02bace'), 'nom': 'Calculatrice scientifique', 'marque': 'Casio', 'stock': 30, 'categorie': 'Électronique', 'prix': 15.0}


Pour les besoins d'un audit, José doit extraire une liste de tous les produits avec leur
prix mis à jour pour inclure une majoration de 5%. Comment peut-il accomplir cela sans
modifier les données dans la base?

In [23]:
mongodb_prix_augmenter = mongodf.copy()
mongodb_prix_augmenter['prix'] = mongodb_prix_augmenter['prix']*1.05
mongodb_prix_augmenter.head(10)


,_id,nom,marque,stock,categorie,prix
0,663908ea39820962ca02bacb,Stylo à bille,Bic,150,Écriture,1.31250
1,663908ea39820962ca02bacc,Cahier A5,Oxford,50,Papeterie,2.62500
2,663908ea39820962ca02bacd,Marqueur permanent,Sharpie,75,Écriture,2.02125
3,663908ea39820962ca02bace,Calculatrice scientifique,Casio,30,Électronique,15.75000
4,663908ea39820962ca02bacf,Pack de 500 feuilles A4,Navigator,150,Papeterie,5.25000
5,663908eb39820962ca02bad0,Ruban adhésif,Scotch,40,Papeterie,3.93750


José veut supprimer définitivement les produits discontinués dont le stock est à 0.
Quelle commande utiliserait-il?

In [24]:
collection_jose.delete_many({'stock':{'$eq':0}})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [25]:
mongodf = pd.DataFrame(list(collection_jose.find()))
mongodf.head(10)

,_id,nom,marque,stock,categorie,prix
0,663908ea39820962ca02bacb,Stylo à bille,Bic,150,Écriture,1.250
1,663908ea39820962ca02bacc,Cahier A5,Oxford,50,Papeterie,2.500
2,663908ea39820962ca02bacd,Marqueur permanent,Sharpie,75,Écriture,1.925
3,663908ea39820962ca02bace,Calculatrice scientifique,Casio,30,Électronique,15.000
4,663908ea39820962ca02bacf,Pack de 500 feuilles A4,Navigator,150,Papeterie,5.000
5,663908eb39820962ca02bad0,Ruban adhésif,Scotch,40,Papeterie,3.750
